# Lambda Code to stop a notebook instance

In [1]:
import boto3
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)


Let's check the SageMaker API and see how can we find the instances that are running in the account

In [10]:
client = boto3.client('sagemaker')
    
candidates = client.list_notebook_instances()
candidates

{'NotebookInstances': [{'NotebookInstanceName': 'fastai-with-efs',
   'NotebookInstanceArn': 'arn:aws:sagemaker:eu-west-1:217431963147:notebook-instance/fastai-with-efs',
   'NotebookInstanceStatus': 'InService',
   'Url': 'fastai-with-efs.notebook.eu-west-1.sagemaker.aws',
   'InstanceType': 'ml.m4.2xlarge',
   'CreationTime': datetime.datetime(2018, 8, 13, 13, 13, 13, 863000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2018, 8, 24, 10, 46, 41, 550000, tzinfo=tzlocal()),
   'NotebookInstanceLifecycleConfigName': 'MountEFS'}],
 'ResponseMetadata': {'RequestId': '3f8ecced-cad7-40ef-8659-42449daf6de4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3f8ecced-cad7-40ef-8659-42449daf6de4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '406',
   'date': 'Fri, 24 Aug 2018 15:31:18 GMT'},
  'RetryAttempts': 0}}

We want to find the name of the instance. It is used to send the stop and start commands

In [11]:
for candidate in candidates['NotebookInstances']:
    candidateName = candidate['NotebookInstanceName']

candidateName

'fastai-with-efs'

To discover the tags of the notebook instances we need to know the ARN of each

In [12]:
instanceArn = candidate['NotebookInstanceArn']
instanceTags = client.list_tags(ResourceArn=instanceArn)
instanceTags['Tags']

[{'Key': 'Demo', 'Value': 'fastai'}, {'Key': 'InDuty', 'Value': 'Yes'}]

We will look for a tag with key of 'InDuty' and a value of 'Yes'

In [33]:
for tag in instanceTags['Tags']:
    if (tag['Key'] == 'InDuty' and tag['Value'] =='Yes'):
        print("Found it")

Now we can connect it all together

In [35]:
client = boto3.client('sagemaker')
    
candidates = client.list_notebook_instances()

for candidate in candidates['NotebookInstances']:
    candidateName = candidate['NotebookInstanceName']
    logger.info('instance {0} is in Service'.format(candidateName))
    instanceArn = candidate['NotebookInstanceArn']
    instanceTags = client.list_tags(ResourceArn=instanceArn)
    for tag in instanceTags['Tags']:
        if (tag['Key'] == 'InDuty' and tag['Value'] =='Yes'):
            logger.info('instance {0} is in duty'.format(candidateName))
    #        response = client.stop_notebook_instance(NotebookInstanceName=candidateName)
            logger.info('instance {0} was stopped'.format(candidateName))
        else:
            logger.info('Not in duty'.format(candidateName))


# Creating a lambda function

Using writefile magic command, we will generate the python file to be uploaded to AWS Lambda. Also if you want to append to file you must use -a parameter.

In [2]:
%%writefile lambda_function.py
import boto3
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

client = boto3.client('sagemaker')

def lambda_handler(event, context):
    
    candidates = client.list_notebook_instances()
    
    for candidate in candidates['NotebookInstances']:
        candidateName = candidate['NotebookInstanceName']
        logger.info('instance {0} is in Service'.format(candidateName))
        instanceArn = candidate['NotebookInstanceArn']
        instanceTags = client.list_tags(ResourceArn=instanceArn)
        for tag in instanceTags['Tags']:
            if (tag['Key'] == 'InDuty' and tag['Value'] =='Yes'):
                logger.info('instance {0} is in duty'.format(candidateName))
                if (event['event'] == "On"):
                    response = client.start_notebook_instance(NotebookInstanceName=candidateName)
                    logger.info('instance {0} was started'.format(candidateName))
                elif (event['event'] == "Off"):
                    response = client.stop_notebook_instance(NotebookInstanceName=candidateName)
                    logger.info('instance {0} was stopped'.format(candidateName))
            else:
                logger.info('Not in duty'.format(candidateName))

    return "Done"


Writing lambda_function.py


Adding the lambda function into a zip file (AWS Lambda requires a zip file)

In [4]:
!zip lambda.zip lambda_function.py

  adding: lambda_function.py (deflated 68%)


We can create the bucket and copy the file to it using the awscli or the python SDK. I'll use the aws cli in this example.

In [6]:
!aws s3 mb s3://terraform-serverless-repository --region=eu-west-1

make_bucket: terraform-serverless-repository


Uploading the zip file to the bucket

In [8]:
!aws s3 cp lambda.zip s3://terraform-serverless-repository/v1.0.0/

upload: ./lambda.zip to s3://terraform-serverless-repository/v1.0.0/lambda.zip


With the source code artifact built and uploaded to S3, we can now write our Terraform configuration to deploy it. In a new directory, create a file named lambda.tf containing the following configuration:

In [63]:
%%writefile lambda.tf
provider "aws" {
  region = "eu-west-1"
}

resource "aws_lambda_function" "start_stop_sm" {
  function_name = "StopStartSageMakerNotebookInstances"

  # The bucket name as created earlier with "aws s3api create-bucket"
  s3_bucket = "terraform-serverless-repository"
  s3_key    = "v1.0.0/lambda.zip"

  # "main" is the filename within the zip file (main.js) and "handler"
  # is the name of the property under which the handler function was
  # exported in that file.
  handler = "lambda_function.lambda_handler"
  runtime = "python3.6"

  role = "${aws_iam_role.lambda_exec.arn}"
}

Overwriting lambda.tf


# Creating IAM Role

In [64]:
%%writefile -a lambda.tf
# IAM role which dictates what other AWS services the Lambda function
# may access.
## Testing a working example
resource "aws_iam_role" "lambda_exec" {
    name = "assume-role"
    assume_role_policy = <<EOF
{
    "Version": "2012-10-17",
    "Statement": [
        {
          "Action": "sts:AssumeRole",
          "Principal": {
            "Service": "lambda.amazonaws.com"
          },
          "Effect": "Allow",
          "Sid": ""
        }
      ]
    }
EOF
}

resource "aws_iam_policy" "policy" {
    name        = "control-policy"
    description = "A policy to control SageMaker instances start/stop state"
    policy = <<EOF
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "CloudWatchLogs0",
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:eu-west-1:217431963147:log-group:/aws/lambda/*NotebookInstance:*"
        },
        {
            "Sid": "InstanceControl",
            "Effect": "Allow",
            "Action": [
                "sagemaker:ListTags",
                "ec2:ModifyNetworkInterfaceAttribute",
                "sagemaker:DescribeNotebookInstance",
                "ec2:DeleteNetworkInterface",
                "ec2:DescribeSecurityGroups",
                "ec2:CreateNetworkInterface",
                "ec2:DescribeInternetGateways",
                "ec2:DescribeNetworkInterfaces",
                "ec2:DescribeAvailabilityZones",
                "sagemaker:StopNotebookInstance",
                "ec2:DescribeVpcs",
                "sagemaker:StartNotebookInstance",
                "ec2:AttachNetworkInterface",
                "ec2:DescribeSubnets",
                "sagemaker:ListNotebookInstances"
            ],
            "Resource": "*"
        },
        {
            "Sid": "CloudWatchLogs1",
            "Effect": "Allow",
            "Action": "logs:CreateLogGroup",
            "Resource": "arn:aws:logs:eu-west-1:217431963147:*"
        }
    ]
}
EOF
}

resource "aws_iam_role_policy_attachment" "test-attach" {
    role       = "${aws_iam_role.lambda_exec.name}"
    policy_arn = "${aws_iam_policy.policy.arn}"
}

Appending to lambda.tf


## Terraform installation

In [21]:
!wget https://releases.hashicorp.com/terraform/0.11.8/terraform_0.11.8_linux_amd64.zip

--2018-08-24 20:39:29--  https://releases.hashicorp.com/terraform/0.11.8/terraform_0.11.8_linux_amd64.zip
Resolving releases.hashicorp.com (releases.hashicorp.com)... 151.101.17.183, 2a04:4e42:4::439
Connecting to releases.hashicorp.com (releases.hashicorp.com)|151.101.17.183|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17871447 (17M) [application/zip]
Saving to: ‘terraform_0.11.8_linux_amd64.zip’

terraform_0.11.8_li 100%[===================>]  17.04M  28.3MB/s    in 0.6s    

2018-08-24 20:39:30 (28.3 MB/s) - ‘terraform_0.11.8_linux_amd64.zip’ saved [17871447/17871447]



In [22]:
!unzip terraform_0.11.8_linux_amd64.zip

Archive:  terraform_0.11.8_linux_amd64.zip
  inflating: terraform               


Before you can work with a new configuration directory, it must be initialized using terraform init, which in this case will install the AWS provider:

In [65]:
!./terraform init


Initializing provider plugins...

The following providers do not have any version constraints in configuration,
so the latest version was installed.

To prevent automatic upgrades to new major versions that may contain breaking
changes, it is recommended to add version = "..." constraints to the
corresponding provider blocks in configuration, with the constraint strings
suggested below.

* provider.aws: version = "~> 1.33"

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


In [66]:
!./terraform validate

In [67]:
!./terraform plan

Refreshing Terraform state in-memory prior to plan...
The refreshed state will be used to calculate this plan, but will not be
persisted to local or remote state storage.

aws_iam_policy.policy: Refreshing state... (ID: arn:aws:iam::217431963147:policy/control-policy)

------------------------------------------------------------------------

An execution plan has been generated and is shown below.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  + aws_iam_role.lambda_exec
      id:                    <computed>
      arn:                   <computed>
      assume_role_policy:    "{\n    \"Version\": \"2012-10-17\",\n    \"Statement\": [\n        {\n          \"Action\": \"sts:AssumeRole\",\n          \"Principal\": {\n            \"Service\": \"lambda.amazonaws.com\"\n          },\n          \"Effect\": \"Allow\",\n          \"Sid\": \"\"\n        }\n      ]\n    }\n"
      create_date:           <computed>
      fo

In [68]:
!./terraform apply -input=false -auto-approve

aws_iam_policy.policy: Refreshing state... (ID: arn:aws:iam::217431963147:policy/control-policy)
aws_iam_role.lambda_exec: Creating...
  arn:                   "" => "<computed>"
  assume_role_policy:    "" => "{\n    \"Version\": \"2012-10-17\",\n    \"Statement\": [\n        {\n          \"Action\": \"sts:AssumeRole\",\n          \"Principal\": {\n            \"Service\": \"lambda.amazonaws.com\"\n          },\n          \"Effect\": \"Allow\",\n          \"Sid\": \"\"\n        }\n      ]\n    }\n"
  create_date:           "" => "<computed>"
  force_detach_policies: "" => "false"
  max_session_duration:  "" => "3600"
  name:                  "" => "assume-role"
  path:                  "" => "/"
  unique_id:             "" => "<computed>"
aws_iam_role.lambda_exec: Creation complete after 0s (ID: assume-role)
aws_iam_role_policy_attachment.test-attach: Creating...
  policy_arn: "" => "arn:aws:iam::217431963147:policy/control-policy"
  role:       "" => "assume-role"
aws_lambda_function

In [69]:
!aws lambda list-functions

{
    "Functions": [
        {
            "FunctionName": "StartNotebookInstance",
            "FunctionArn": "arn:aws:lambda:eu-west-1:217431963147:function:StartNotebookInstance",
            "Runtime": "python3.6",
            "Role": "arn:aws:iam::217431963147:role/service-role/StartStopSageMakerInstances",
            "Handler": "lambda_function.lambda_handler",
            "CodeSize": 542,
            "Description": "",
            "Timeout": 3,
            "MemorySize": 128,
            "LastModified": "2018-08-24T14:36:50.195+0000",
            "CodeSha256": "cHgIPdy3EnqRNIg1/vRo64u+aDzCUdcbU7Bi4OzSsdw=",
            "Version": "$LATEST",
            "VpcConfig": {
                "SubnetIds": [],
                "SecurityGroupIds": [],
                "VpcId": ""
            },
            "TracingConfig": {
                "Mode": "PassThrough"
            },
            "RevisionId": "a9f98143-d897-4b99-b001-31e85f56840b"
        },
        {
            "FunctionName": "St

In [77]:
!aws lambda invoke --function-name StopStartSageMakerNotebookInstances \
    --payload '{"event":"On"}' \
    --invocation-type RequestResponse \
    --log-type Tail /dev/null | jq -r '.LogResult' | base64 --decode

START RequestId: 4e8c3655-a872-11e8-a36c-296fe0514407 Version: $LATEST
[INFO]	2018-08-25T14:22:33.251Z	4e8c3655-a872-11e8-a36c-296fe0514407	Resetting dropped connection: sagemaker.eu-west-1.amazonaws.com
[INFO]	2018-08-25T14:22:33.411Z	4e8c3655-a872-11e8-a36c-296fe0514407	instance fastai-with-efs is in Service
[INFO]	2018-08-25T14:22:33.520Z	4e8c3655-a872-11e8-a36c-296fe0514407	Not in duty
[INFO]	2018-08-25T14:22:33.522Z	4e8c3655-a872-11e8-a36c-296fe0514407	instance fastai-with-efs is in duty
An error occurred (ValidationException) when calling the StartNotebookInstance operation: Status (InService) not in ([Stopped, Failed]). Unable to transition to (Pending) for Notebook Instance (arn:aws:sagemaker:eu-west-1:217431963147:notebook-instance/fastai-with-efs).: ClientError
Traceback (most recent call last):
  File "/var/task/lambda_function.py", line 22, in lambda_handler
    response = client.start_notebook_instance(NotebookInstanceName=candidateName)
  File "/var/runtime/botocore/clien

## Adding CloudWatch events

We will create two CloudWatch events, one for the start of the working day and one for its end. They will run using cron that we will set the schedule of the office that we cover with it. Both events will call the same Lambda function that we defined above, but each will send a different constant event, one with "On" and the other with "Off"

In [91]:
%%writefile cloud_watch.tf
## Based on the location of the instances. This is for Israel where 
## people are working Sunday to Thursday and 5AM GTM is 8AM
resource "aws_cloudwatch_event_rule" "on_duty" {
    name = "on_duty"
    description = "Fires at the beginning of the working day"
    schedule_expression = "cron(0 5 ? * SUN-THU *)"
}

## people are working Sunday to Thursday and 4PM GTM is 7PM
resource "aws_cloudwatch_event_rule" "off_duty" {
    name = "off_duty"
    description = "Fires at the end of the working day"
    schedule_expression = "cron(0 16 ? * SUN-THU *)"
}

resource "aws_cloudwatch_event_target" "on_duty_start_instances" {
    rule = "${aws_cloudwatch_event_rule.on_duty.name}"
    target_id = "start_stop_sm"
    arn = "${aws_lambda_function.start_stop_sm.arn}"
    input = <<DOC
{
  "event": "On"
}
DOC
}

resource "aws_cloudwatch_event_target" "off_duty_start_instances" {
    rule = "${aws_cloudwatch_event_rule.off_duty.name}"
    target_id = "start_stop_sm"
    arn = "${aws_lambda_function.start_stop_sm.arn}"
    input = <<DOC
{
  "event": "Off"
}
DOC
}

resource "aws_lambda_permission" "allow_cloudwatch_to_call_On_duty" {
    statement_id = "AllowExecutionFromCloudWatchOn"
    action = "lambda:InvokeFunction"
    function_name = "${aws_lambda_function.start_stop_sm.function_name}"
    principal = "events.amazonaws.com"
    source_arn = "${aws_cloudwatch_event_rule.on_duty.arn}"
}

resource "aws_lambda_permission" "allow_cloudwatch_to_call_Off_duty" {
    statement_id = "AllowExecutionFromCloudWatchOff"
    action = "lambda:InvokeFunction"
    function_name = "${aws_lambda_function.start_stop_sm.function_name}"
    principal = "events.amazonaws.com"
    source_arn = "${aws_cloudwatch_event_rule.off_duty.arn}"
}

Overwriting cloud_watch.tf


Now we can update our environment with the new configuration using Terraform apply command

In [92]:
!./terraform apply -input=false -auto-approve

aws_iam_policy.policy: Refreshing state... (ID: arn:aws:iam::217431963147:policy/control-policy)
aws_iam_role.lambda_exec: Refreshing state... (ID: assume-role)
aws_cloudwatch_event_rule.on_duty: Refreshing state... (ID: on_duty)
aws_lambda_function.start_stop_sm: Refreshing state... (ID: StopStartSageMakerNotebookInstances)
aws_lambda_permission.allow_cloudwatch_to_call_On_duty: Refreshing state... (ID: AllowExecutionFromCloudWatch)
aws_cloudwatch_event_target.on_duty_start_instances: Refreshing state... (ID: on_duty-start_stop_sm)
aws_iam_role_policy_attachment.test-attach: Refreshing state... (ID: assume-role-20180825141001678900000001)
aws_lambda_permission.allow_cloudwatch_to_call_On_duty: Destroying... (ID: AllowExecutionFromCloudWatch)
aws_cloudwatch_event_rule.off_duty: Creating...
  arn:                 "" => "<computed>"
  description:         "" => "Fires at the end of the working day"
  is_enabled:          "" => "true"
  name:                "" => "off_duty"
  schedule_exp